# Czas koncentracji zlewni

Czas koncentracji (tc) to czas potrzebny na przepływ wody z najdalszego punktu zlewni do przekroju zamykającego.

Hydrolog oferuje trzy metody obliczania tc:
1. **Kirpich** - najpopularniejsza, dla małych zlewni
2. **NRCS** - uwzględnia CN (pokrycie terenu)
3. **Giandotti** - dla większych zlewni górskich

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))

from hydrolog.time import ConcentrationTime

## Przykładowa zlewnia

Parametry zlewni do obliczeń:

In [ ]:
# Parametry zlewni
area_km2 = 45.0          # Powierzchnia [km²]
length_km = 12.0         # Długość cieku głównego [km]
length_m = 12000         # Długość cieku głównego [m]
elevation_diff_m = 350   # Różnica wysokości [m]
slope_m_per_m = elevation_diff_m / length_m  # Spadek [-]
slope_percent = slope_m_per_m * 100          # Spadek [%]
cn = 72                  # Curve Number

print("PARAMETRY ZLEWNI")
print("=" * 40)
print(f"Powierzchnia: {area_km2} km²")
print(f"Długość cieku: {length_km} km ({length_m} m)")
print(f"Różnica wysokości: {elevation_diff_m} m")
print(f"Spadek: {slope_m_per_m:.4f} m/m ({slope_percent:.2f}%)")
print(f"CN: {cn}")

## 1. Wzór Kirpicha

Najpopularniejsza metoda, opracowana dla małych zlewni rolniczych.

$$t_c = 0.0195 \cdot L^{0.77} \cdot S^{-0.385}$$

Gdzie:
- L - długość cieku [m]
- S - spadek cieku [m/m]

**Uwaga:** W Hydrolog podajemy L w km, a funkcja przelicza jednostki.

In [ ]:
# Wzór Kirpicha
tc_kirpich = ConcentrationTime.kirpich(
    length_km=length_km,
    slope_m_per_m=slope_m_per_m
)

print(f"Czas koncentracji (Kirpich): {tc_kirpich:.1f} min ({tc_kirpich/60:.2f} h)")

## 2. Wzór NRCS

Metoda NRCS uwzględnia Curve Number, co pozwala na lepsze odwzorowanie wpływu użytkowania terenu.

$$t_c = 0.01416 \cdot L^{0.8} \cdot (S_{ret}+25.4)^{0.7} \cdot Y^{-0.5}$$

Gdzie:
- L - długość hydrauliczna [m]
- S_ret = (25400/CN) - 254 - retencja [mm]
- Y - spadek [%]

In [ ]:
# Wzór NRCS
tc_nrcs = ConcentrationTime.nrcs(
    length_km=length_km,
    slope_m_per_m=slope_m_per_m,
    cn=cn
)

print(f"Czas koncentracji (NRCS): {tc_nrcs:.1f} min ({tc_nrcs/60:.2f} h)")

## 3. Wzór Giandottiego

Metoda opracowana dla zlewni górskich we Włoszech. Uwzględnia powierzchnię zlewni i średnią wysokość.

$$t_c = \frac{4\sqrt{A} + 1.5L}{0.8\sqrt{H_m}}$$

Gdzie:
- A - powierzchnia zlewni [km²]
- L - długość cieku głównego [km]
- H_m - różnica wysokości (deniwelacja) [m]

In [ ]:
# Wzór Giandottiego
tc_giandotti = ConcentrationTime.giandotti(
    area_km2=area_km2,
    length_km=length_km,
    elevation_diff_m=elevation_diff_m
)

print(f"Czas koncentracji (Giandotti): {tc_giandotti:.1f} min ({tc_giandotti/60:.2f} h)")

## 4. Porównanie metod

In [ ]:
print("PORÓWNANIE METOD")
print("=" * 50)
print(f"{'Metoda':<20} {'tc [min]':>12} {'tc [h]':>12}")
print("-" * 50)
print(f"{'Kirpich':<20} {tc_kirpich:>12.1f} {tc_kirpich/60:>12.2f}")
print(f"{'NRCS':<20} {tc_nrcs:>12.1f} {tc_nrcs/60:>12.2f}")
print(f"{'Giandotti':<20} {tc_giandotti:>12.1f} {tc_giandotti/60:>12.2f}")
print("-" * 50)

# Średnia i rozstęp
tc_values = [tc_kirpich, tc_nrcs, tc_giandotti]
tc_mean = sum(tc_values) / len(tc_values)
tc_range = max(tc_values) - min(tc_values)

print(f"{'Średnia':<20} {tc_mean:>12.1f} {tc_mean/60:>12.2f}")
print(f"Rozstęp: {tc_range:.1f} min ({tc_range/tc_mean*100:.1f}% średniej)")

## 5. Wpływ parametrów na tc

In [ ]:
# Wpływ długości zlewni (Kirpich)
print("Wpływ długości zlewni (Kirpich, S=0.03):")
print("-" * 40)
for L in [2, 5, 10, 15, 20]:
    tc = ConcentrationTime.kirpich(length_km=L, slope_m_per_m=0.03)
    print(f"L = {L:2d} km  →  tc = {tc:6.1f} min")

In [ ]:
# Wpływ spadku (Kirpich)
print("\nWpływ spadku (Kirpich, L=10 km):")
print("-" * 40)
for S in [0.01, 0.02, 0.03, 0.05, 0.10]:
    tc = ConcentrationTime.kirpich(length_km=10, slope_m_per_m=S)
    print(f"S = {S:.2f} m/m ({S*100:.0f}%)  →  tc = {tc:6.1f} min")

In [ ]:
# Wpływ CN (NRCS)
print("\nWpływ CN (NRCS, L=10km, S=0.03):")
print("-" * 40)
for CN in [50, 60, 70, 80, 90]:
    tc = ConcentrationTime.nrcs(length_km=10, slope_m_per_m=0.03, cn=CN)
    print(f"CN = {CN}  →  tc = {tc:6.1f} min")

## 6. Wizualizacja

In [ ]:
try:
    import matplotlib.pyplot as plt
    import numpy as np
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Wpływ długości
    lengths = np.linspace(1, 25, 50)
    tc_vals = [ConcentrationTime.kirpich(L, 0.03) for L in lengths]
    axes[0].plot(lengths, tc_vals, 'b-', linewidth=2)
    axes[0].set_xlabel('Długość cieku [km]')
    axes[0].set_ylabel('Czas koncentracji [min]')
    axes[0].set_title('Wpływ długości (Kirpich, S=3%)')
    axes[0].grid(True, alpha=0.3)
    
    # Wpływ spadku
    slopes = np.linspace(0.005, 0.15, 50)
    tc_vals = [ConcentrationTime.kirpich(10, S) for S in slopes]
    axes[1].plot(slopes * 100, tc_vals, 'g-', linewidth=2)
    axes[1].set_xlabel('Spadek [%]')
    axes[1].set_ylabel('Czas koncentracji [min]')
    axes[1].set_title('Wpływ spadku (Kirpich, L=10km)')
    axes[1].grid(True, alpha=0.3)
    
    # Wpływ CN
    cns = np.linspace(40, 95, 50)
    tc_vals = [ConcentrationTime.nrcs(length_km=10, slope_m_per_m=0.03, cn=int(cn)) for cn in cns]
    axes[2].plot(cns, tc_vals, 'r-', linewidth=2)
    axes[2].set_xlabel('Curve Number (CN)')
    axes[2].set_ylabel('Czas koncentracji [min]')
    axes[2].set_title('Wpływ CN (NRCS, L=10km, S=3%)')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Matplotlib nie jest zainstalowany.")

## Podsumowanie

| Metoda | Parametry wejściowe | Zastosowanie |
|--------|--------------------|--------------|
| Kirpich | L, S | Małe zlewnie rolnicze (<80 km²) |
| NRCS | L, S, CN | Zlewnie z różnym pokryciem terenu |
| Giandotti | A, L, H | Większe zlewnie górskie |

**Zalecenia:**
- Dla małych zlewni miejskich/rolniczych: **Kirpich**
- Gdy znany jest CN: **NRCS**
- Dla zlewni górskich: **Giandotti**
- W razie wątpliwości: użyj średniej z kilku metod